![](Capture.PNG)

In [48]:
from itertools import cycle

import random
import hashlib

In [49]:
def is_prime(n, k=128):
    # Implementing Miller-Rabin primality test
    if n <= 1 or n == 4:
        return False
    if n <= 3:
        return True
    
    d = n - 1
    while d % 2 == 0:
        d //= 2
    
    for _ in range(k):
        if not miller_test(d, n):
            return False
    return True

def miller_test(d, n):
    a = 2 + random.randint(1, n - 4)
    x = pow(a, d, n)
    if x == 1 or x == n - 1:
        return True
    
    while d != n - 1:
        x = (x * x) % n
        d *= 2
        
        if x == 1:
            return False
        if x == n - 1:
            return True
    return False

def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

def generate_prime_candidate(length):
    p = random.getrandbits(length)
    p |= (1 << length - 1) | 1  # Ensure p is odd and has the length we want
    return p

def generate_prime_number(length=1024):
    p = 4  # Initialize with a non-prime
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p

def int_to_bytes(i):
    return i.to_bytes((i.bit_length() + 7) // 8, byteorder='big')

def bytes_to_int(b):
    return int.from_bytes(b, byteorder='big')
def generate_large_prime(length):
    # Generate a large prime number of the specified bit length
    while True:
        num = random.getrandbits(length)
        if is_prime(num):
            return num

def modexp(base, exponent, modulus):
    # Modular exponentiation function to compute (base^exponent) % modulus efficiently
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

AES IMPLiMENTION:


In [50]:
def dummy_aes_block_encrypt(block, key):
    # Dummy block "encryption" for demonstration purposes.
    # In practice, this should be the actual AES encryption operation.
    return bytes([b ^ k for b, k in zip(block, cycle(key))])

def xor_bytes(block1, block2):
    return bytes([b1 ^ b2 for b1, b2 in zip(block1, block2)])

def increment_nonce(nonce):
    nonce_as_int = int.from_bytes(nonce, byteorder='big')
    incremented_nonce = nonce_as_int + 1
    return incremented_nonce.to_bytes(16, byteorder='big')

def aes_ctr_decrypt(ciphertext, key, nonce):
    # Split the ciphertext into 16-byte blocks
    ciphertext_blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    
    decrypted_message = b''
    for block in ciphertext_blocks:
        # Encrypt the nonce (as a counter block), which is the same as decryption in CTR
        encrypted_nonce = dummy_aes_block_encrypt(nonce, key)
        
        # XOR the encrypted nonce with the block
        decrypted_block = xor_bytes(encrypted_nonce, block)
        
        # Append decrypted block to the decrypted message
        decrypted_message += decrypted_block
        
        # Increment the nonce for the next block
        nonce = increment_nonce(nonce)
    
    return decrypted_message
def dummy_aes_block_cipher(block, key):
    # This should be the actual AES block encryption/decryption.
    # Here we are just using a dummy XOR for demonstration.
    key_gen = cycle(key)
    return bytes([b ^ next(key_gen) for b in block])

def aes_ctr_decrypt(ciphertext, key, nonce):
    # Split the ciphertext into 16-byte blocks
    ciphertext_blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    
    decrypted_message = b''
    for block in ciphertext_blocks:
        # Encrypt the nonce (as a counter block), which is the same as decryption in CTR
        encrypted_nonce = dummy_aes_block_cipher(nonce, key)
        
        # XOR the encrypted nonce with the block
        decrypted_block = xor_bytes(encrypted_nonce, block)
        
        # Append decrypted block to the decrypted message
        decrypted_message += decrypted_block
        
        # Increment the nonce for the next block
        nonce = increment_nonce(nonce)
    
    return decrypted_message

def dummy_aes_block_decrypt(block, key):
    # In CTR mode, block decryption is identical to block encryption
    return dummy_aes_block_encrypt(block, key)

def aes_ctr_encrypt(message, key, nonce):
    # Split the message into 16-byte blocks
    message_blocks = [message[i:i+16] for i in range(0, len(message), 16)]
    
    encrypted_message = b''
    for block in message_blocks:
        # Encrypt the nonce (as a counter block)
        encrypted_nonce = dummy_aes_block_encrypt(nonce, key)
        
        # XOR the encrypted nonce with the block
        block = block.ljust(16, b'\0')  # Padding if block is less than 16 bytes
        encrypted_block = xor_bytes(encrypted_nonce, block)
        
        # Append encrypted block to the encrypted message
        encrypted_message += encrypted_block
        
        # Increment the nonce for the next block
        nonce = increment_nonce(nonce)
    
    return encrypted_message


def aes_ctr_decrypt(ciphertext, key, nonce):
    # Split the ciphertext into 16-byte blocks
    ciphertext_blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    
    decrypted_message = b''
    for block in ciphertext_blocks:
        # Decrypt the nonce (as a counter block), which is the same as encryption in CTR
        decrypted_nonce = dummy_aes_block_decrypt(nonce, key)
        
        # XOR the decrypted nonce with the block
        decrypted_block = xor_bytes(decrypted_nonce, block)
        
        # Append decrypted block to the decrypted message
        decrypted_message += decrypted_block
        
        # Increment the nonce for the next block
        nonce = increment_nonce(nonce)
    
    return decrypted_message


RSA :

In [51]:
def generate_rsa_keys(keysize=2048):
    p = generate_prime_number(keysize // 2)
    q = generate_prime_number(keysize // 2)
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = 65537
    d = modinv(e, phi)
    
    # Return public and private keypair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt_message_with_rsa(m, public_key):
    e, n = public_key
    if m < 0 or m >= n:
        raise ValueError("Message must be a non-negative integer less than the modulus")

    c = pow(m, e, n)
    return c

def decrypt_message_with_rsa(c, private_key):
    d, n = private_key
    if c < 0 or c >= n:
        raise ValueError("Ciphertext must be a non-negative integer less than the modulus")
    
    # Decryption formula
    m = pow(c, d, n)
    return m  # Add this line to return the decrypted message

def encrypt_aes_key_with_rsa(aes_key, rsa_public_key):
    # Convert the AES key from bytes to an int
    aes_key_int = bytes_to_int(aes_key)
    
    # Encrypt it with the RSA public key
    encrypted_key_int = encrypt_message_with_rsa(aes_key_int, rsa_public_key)
    
    # Convert back to bytes
    encrypted_aes_key = int_to_bytes(encrypted_key_int)
    
    return encrypted_aes_key

from Crypto.Random import get_random_bytes

def decrypt_aes_key_with_rsa(encrypted_aes_key, rsa_private_key):
    # Convert the encrypted AES key from bytes to an int
    encrypted_key_int = bytes_to_int(encrypted_aes_key)
    
    # Decrypt it with the RSA private key
    decrypted_key_int = decrypt_message_with_rsa(encrypted_key_int, rsa_private_key)
    
    # Convert back to bytes
    decrypted_aes_key = int_to_bytes(decrypted_key_int)
    
    return decrypted_aes_key

EL-GAMAL Implementions :

In [52]:
def elgamal_generate_keys(p):
    g = 2
    x = random.randint(1, p - 1)
    h = pow(g, x, p)
    return g, h, x  # Public key: (p, g, h), private key: x

def elgamal_encrypt(msg, p, g, h):
    y = random.randint(1, p - 1)
    c1 = pow(g, y, p)
    s = pow(h, y, p)
    c2 = (msg * s) % p
    return c1, c2

def elgamal_decrypt(c1, c2, p, x):
    s = pow(c1, x, p)
    m = (c2 * pow(s, p - 2, p)) % p  # Using Fermat's little theorem for modular inverse
    return m

In [53]:
def simulate_conversation_rsa(message_list_a, message_list_b):
    privkey_A, pubkey_A = generate_rsa_keys()
    privkey_B, pubkey_B = generate_rsa_keys()
    
    # A creates AES key and nonce for the session
    aes_key = get_random_bytes(16)
    nonce = get_random_bytes(8)  # 64-bit nonce for AES CTR
    
    # A encrypts AES key with B's public RSA key
    encrypted_aes_key = encrypt_aes_key_with_rsa(aes_key, pubkey_B)
    
    # B decrypts the AES key with their private RSA key
    decrypted_aes_key = decrypt_aes_key_with_rsa(encrypted_aes_key, privkey_B)
    
    # Now both A and B have the same AES key and nonce
    
    # Begin conversation - A starts
    for i in range(max(len(message_list_a), len(message_list_b))):
        if i < len(message_list_a):
            # A sends a message to B
            encrypted_message = aes_ctr_encrypt(message_list_a[i][0].encode('utf-8'), aes_key, nonce)
            print("A sent an encrypted message:", encrypted_message)
            
            # B receives and decrypts the message
            try:
                decrypted_message = aes_ctr_decrypt(encrypted_message, decrypted_aes_key, nonce)
                print("B received and decrypted the message:", decrypted_message.decode('utf-8'))
            except UnicodeDecodeError:
                print("B received an encrypted or corrupted message that cannot be decoded.")
        
        if i < len(message_list_b):
            # B sends a message to A
            encrypted_message = aes_ctr_encrypt(message_list_b[i][0].encode('utf-8'), decrypted_aes_key, nonce)
            print("B sent an encrypted message:", encrypted_message)
            
            # A receives and decrypts the message
            try:
                decrypted_message = aes_ctr_decrypt(encrypted_message, aes_key, nonce)
                print("A received and decrypted the message:", decrypted_message.decode('utf-8'))
            except UnicodeDecodeError:
                print("A received an encrypted or corrupted message that cannot be decoded.")

# Define messages
messageA = [["hello"], ["tnks"], ["good bye"]]
messageB = [["hi!"], ["whats up"], ["give me flower"]]

# Start the conversation simulation


<h2>DH implimention</h2
>

In [54]:

def diffie_hellman_generate_keys(p, g):
    private_key = random.randint(1, p - 2)  # A random number, private key
    public_key = pow(g, private_key, p)  # g^private_key mod p
    return private_key, public_key

def diffie_hellman_compute_shared_secret(other_public_key, private_key, p):
    # Compute the shared secret key
    shared_secret = modexp(other_public_key, private_key, p)
    return shared_secret


p = generate_prime_number(2048)  # Replace this with your actual function call if different
g = 2
a_private, a_public = diffie_hellman_generate_keys(p, g)
b_private, b_public = diffie_hellman_generate_keys(p, g)

# Compute the shared secret for both parties, again
a_shared_secret = diffie_hellman_compute_shared_secret(b_public, a_private, p)
b_shared_secret = diffie_hellman_compute_shared_secret(a_public, b_private, p)

# Ensure that both shared secrets are equal
shared_secret_match = a_shared_secret == b_shared_secret

# Output results for verification
a_private, a_public, b_private, b_public, shared_secret_match



def diffie_hellman_key_exchange():
    # User A generates their keys
    a_private, a_public, p, g = diffie_hellman_generate_keys()
    # User B generates their keys
    b_private, b_public, _, _ = diffie_hellman_generate_keys(prime_length=1024)
    
    # Exchange public keys and compute shared secret
    a_shared_secret = diffie_hellman_compute_shared_secret(b_public, a_private, p)
    b_shared_secret = diffie_hellman_compute_shared_secret(a_public, b_private, p)
    
    # Verify that the shared secrets are the same
    assert a_shared_secret == b_shared_secret
    
    # The shared secret can be used as the AES key for further encryption
    aes_key = a_shared_secret.to_bytes((a_shared_secret.bit_length() + 7) // 8, 'big')[:16]  # Truncate to 128 bits
    return aes_key

def derive_aes_key_from_shared_secret(shared_secret):
    # Hash the shared secret to derive a 256-bit key and take the first 128 bits
    hashed_secret = hashlib.sha256(int.to_bytes(shared_secret, shared_secret.bit_length() // 8 + 1, 'big')).digest()
    return hashed_secret[:16]

# Derive AES keys from the shared secret for both parties (which should be the same)
aes_key_a = derive_aes_key_from_shared_secret(a_shared_secret)
aes_key_b = derive_aes_key_from_shared_secret(b_shared_secret)

# Verify that both AES keys are the same
aes_key_match = aes_key_a == aes_key_b

In [55]:
def elgamal_simulate_conversation(message_list_a, message_list_b):
    # ElGamal key generation
    p = generate_prime_number(2048)
    g_A, h_A, x_A = elgamal_generate_keys(p)  # A's keys
    g_B, h_B, x_B = elgamal_generate_keys(p)  # B's keys
    
    # Simulate conversation
    for i in range(max(len(message_list_a), len(message_list_b))):
        # User A sends a message to User B
        if i < len(message_list_a):
            message = int.from_bytes(message_list_a[i][0].encode(), 'big')  # Convert message to int
            c1, c2 = elgamal_encrypt(message, p, g_B, h_B)  # A encrypts using B's public key
            print(f"A sent an encrypted message: (c1={c1}, c2={c2})")
            
            decrypted_message = elgamal_decrypt(c1, c2, p, x_B)  # B decrypts using their private key
            decrypted_message_bytes = decrypted_message.to_bytes((decrypted_message.bit_length() + 7) // 8, 'big')
            print(f"B decrypted the message: {decrypted_message_bytes.decode()}")

        # User B sends a message to User A
        if i < len(message_list_b):
            message = int.from_bytes(message_list_b[i][0].encode(), 'big')  # Convert message to int
            c1, c2 = elgamal_encrypt(message, p, g_A, h_A)  # B encrypts using A's public key
            print(f"B sent an encrypted message: (c1={c1}, c2={c2})")
            
            decrypted_message = elgamal_decrypt(c1, c2, p, x_A)  # A decrypts using their private key
            decrypted_message_bytes = decrypted_message.to_bytes((decrypted_message.bit_length() + 7) // 8, 'big')
            print(f"A decrypted the message: {decrypted_message_bytes.decode()}")


In [56]:
def diffie_hellman_generate_keys(p, g):
    private_key = random.randint(2, p-2)  # Generate a private key
    public_key = pow(g, private_key, p)  # Compute the public key
    return private_key, public_key

def derive_aes_key_from_shared_secret(shared_secret):
    # Hash the shared secret to derive a 256-bit key and take the first 128 bits
    hashed_secret = hashlib.sha256(int.to_bytes(shared_secret, shared_secret.bit_length() // 8 + 1, 'big')).digest()
    return hashed_secret[:16]

# Derive AES keys from the shared secret for both parties (which should be the same)
aes_key_a = derive_aes_key_from_shared_secret(a_shared_secret)
aes_key_b = derive_aes_key_from_shared_secret(b_shared_secret)

# Verify that both AES keys are the same
aes_key_match = aes_key_a == aes_key_b

# Now let's define the conversation simulation using the derived AES key
def simulate_conversation_dh(message_list_a, message_list_b, aes_key, nonce):
    # Begin conversation
    for i in range(max(len(message_list_a), len(message_list_b))):
        if i < len(message_list_a):
            # A sends a message to B
            encrypted_message = aes_ctr_encrypt(message_list_a[i][0].encode('utf-8'), aes_key, nonce)
            print("A sent an encrypted message:", encrypted_message)
            
            # B receives and decrypts the message
            decrypted_message = aes_ctr_decrypt(encrypted_message, aes_key, nonce)
            print("B received and decrypted the message:", decrypted_message.decode('utf-8'))
        
        if i < len(message_list_b):
            # B sends a message to A
            encrypted_message = aes_ctr_encrypt(message_list_b[i][0].encode('utf-8'), aes_key, nonce)
            print("B sent an encrypted message:", encrypted_message)
            
            # A receives and decrypts the message
            decrypted_message = aes_ctr_decrypt(encrypted_message, aes_key, nonce)
            print("A received and decrypted the message:", decrypted_message.decode('utf-8'))

# Generate a nonce for the AES CTR


In [57]:
messageA = [["hello"], ["thanks"], ["goodbye"]]
messageB = [["hi"], ["welcome"], ["see you"]]

In [58]:
elgamal_simulate_conversation(messageA, messageB)

A sent an encrypted message: (c1=16286866731434143520920581777583474568237280778739342271005738673361897322054831575557726011528701334546639439515769318087100302527376186954521281269411860156424897444527597515442683155830210688360984264104178228907406004193083810185258216320320759368957425325319638710310915013150121357633516308243003658283154623406694922607351770230226371566180737866892789219659708221005691285320064434802031256672354602360673526172964247391753699141027023147342186476660366254665420684549466884366064031056959081058460900092406879391868068300678782140328009236105634217615769417470421468461832843343516153881771636119121682361907, c2=114270676280744005959501853554102812168554731097358314890142441950073499632135750703205133347486955635875986357246735507498132634425093691536085984896824056467988808189101914602978725192988767357178138337010091984623214892954981734930952303107487975522027282099000646831412794842340171042013934718609332048893705240066264709222816324243487338342

In [59]:
simulate_conversation_rsa(messageA, messageB)

A sent an encrypted message: b'qu\xea\x05_IP\x04'
B received and decrypted the message: hello   
B sent an encrypted message: b'qy\x86i0IP\x04'
A received and decrypted the message: hi      
A sent an encrypted message: b'mx\xe7\x07[:P\x04'
B received and decrypted the message: thanks  
B sent an encrypted message: b'nu\xea\n_$5\x04'
A received and decrypted the message: welcome 
A sent an encrypted message: b'~\x7f\xe9\rR05\x04'
B received and decrypted the message: goodbye 
B sent an encrypted message: b'ju\xe3II&%\x04'
A received and decrypted the message: see you 


In [60]:
nonce = get_random_bytes(8)  # 64-bit nonce
simulate_conversation_dh(messageA, messageB, aes_key_a, nonce)

A sent an encrypted message: b'\xefKC\xf3$9\x13\x0e'
B received and decrypted the message: hello   
B sent an encrypted message: b'\xefG/\x9fK9\x13\x0e'
A received and decrypted the message: hi      
A sent an encrypted message: b'\xf3FN\xf1 J\x13\x0e'
B received and decrypted the message: thanks  
B sent an encrypted message: b'\xf0KC\xfc$Tv\x0e'
A received and decrypted the message: welcome 
A sent an encrypted message: b'\xe0A@\xfb)@v\x0e'
B received and decrypted the message: goodbye 
B sent an encrypted message: b'\xf4KJ\xbf2Vf\x0e'
A received and decrypted the message: see you 
